In [ ]:
import pandas as pd
import spotipy
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt 
from azure.storage.blob import BlobServiceClient

In [ ]:
connection_string = "YOUR_CONNECTION_STRING"
container_name = "CONTAINER_NAME"

blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
blob_name = "music_data.csv"
if container_client.exists():
    blob_client = container_client.get_blob_client(blob_name)
    with open('music_data.csv', "wb") as my_blob:
        blob_data = blob_client.download_blob()
        my_blob.write(blob_data.readall())

else:
    print(f"Container '{container_name}' does not exist.")

In [ ]:
df = pd.read_csv("music_data.csv")

In [ ]:
df 

In [ ]:
df.info()

In [ ]:
df['artist_name'] = df['artist_name'].astype(str)
df['artist_id'] = df['artist_id'].astype(str)
df['first_added'] = pd.to_datetime(df['first_added'])
df['last_added'] = pd.to_datetime(df['last_added'])

In [ ]:
df.info()

In [ ]:
df

In [ ]:
years = sorted(df['first_added'].dt.year.unique())
print(years)

In [ ]:
one_song_liked = df[df['amount_of_liked_songs'] == 1]
one_song_liked

In [ ]:
for year in years:
    print(f"{year} : {len(one_song_liked[one_song_liked['first_added'].dt.year == year])}")